In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Decision trees
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
#import graphviz
import os

In [27]:
# Set up the data that we want to create a DT on.
# import the fx data , econ and value data for EURUSD.
# then create the features (on trend and econ data) standardise and run a DT on the x_train sample.
# what is  target? 1 day ahead or long days ahead? trade on binary data.
csv_file = {"FXData" : r"C:\Users\edgil\Documents\Masters\dissertation\data\CurrencyData.csv",
            "ValueData" : r"",
            "EconData" : r"",
            }
fxdata = pd.read_csv(csv_file["FXData"])

In [29]:
# use this to make sure the dates are imported correctly as UK dates
fxdata['Date'] = pd.to_datetime(fxdata['Date'], format= '%d/%m/%Y %H:%M')
# Separate out the EURUSD factor
eurusd = fxdata[["Date", "EURUSD"]]


In [40]:
# Build out the featur set on price, this may need to be created using funcional process.
eurusd["logret"] = np.log(eurusd["EURUSD"]) - np.log(eurusd["EURUSD"].shift(1))
# Standardising the daily rets and accumulating the standardised returns, or should we sum the % ret and standardise by its own history
# is difference between different accumulated retusn horizons the same as the macd?
# should we standardise by the 1 year forward vol?
lkbk1M = 22
lkbk3M = 66
lkbk6M = 132
lkbk9M = 198
eurusd['1MRet'] = eurusd["logret"].rolling(lkbk1M).sum()
eurusd['3MRet'] = eurusd["logret"].rolling(lkbk3M).sum()
eurusd['6MRet'] = eurusd["logret"].rolling(lkbk6M).sum()
eurusd['9MRet'] = eurusd["logret"].rolling(lkbk9M).sum()
eurusd['1Mv3MRet'] = eurusd["1MRet"].sub(eurusd["3MRet"])
eurusd['1Mv6MRet'] = eurusd["1MRet"].sub(eurusd["6MRet"])

C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [51]:
# Standardise the data using sklearn
from sklearn.preprocessing import StandardScaler
# Very important step is to truncate the data so that we do not see the last 1 year of data.
# Q. should we not have a rollign window type of model? or just always aggregate the data from the start?
# how ong is testing? we should we the train and test to sizes which make sense to the type of model we use going forward.
eurusd = eurusd.loc[eurusd['Date'] < "2018-01-01 00:00"]
# train size should be at least 5 years?
eurusd_train = eurusd.loc[eurusd['Date'] < "2010-01-01 00:00"]
eurusd_test = eurusd.loc[eurusd['Date'] > "2010-02-01 00:00"]

                      Date   EURUSD    logret     1MRet     3MRet     6MRet  \
112675 2017-12-29 19:00:00  1.20140 -0.000666  0.006179  0.012648  0.013154   
112676 2017-12-29 20:00:00  1.20095 -0.000375  0.005553  0.012442  0.013117   
112677 2017-12-29 21:00:00  1.19980 -0.000958  0.004971  0.011231  0.011990   
112678 2017-12-29 22:00:00  1.20035  0.000458  0.005346  0.011690  0.012955   
112679 2017-12-29 23:00:00  1.20035  0.000000  0.004885  0.010763  0.013039   

           9MRet  1Mv3MRet  1Mv6MRet  
112675  0.015518 -0.006470 -0.006976  
112676  0.015313 -0.006890 -0.007564  
112677  0.015793 -0.006260 -0.007019  
112678  0.016252 -0.006344 -0.007609  
112679  0.015236 -0.005877 -0.008154  


In [ ]:
X_test, Y_test = X_test_norm, Y_test # we need to change the X_test to pca version 
                                     # if using that (X_test_norm,principalDf_test, X_test_norm
X , Y = X_train, y_train # set X_train = principalDf
#### This is where we choose and set up the models!
RF = RandomForestClassifier(n_estimators=150,max_features=5)
RF.fit(X, Y)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

In [43]:
eurusd = eurusd.replace(np.nan, 0)
print(eurusd.tail(25))

                      Date   EURUSD    logret     1MRet     3MRet     6MRet  \
121055 2019-05-02 23:00:00  1.11720  0.000000 -0.002682 -0.001431  0.003407   
121056 2019-05-03 00:00:00  1.11750  0.000268 -0.002770 -0.001252  0.003765   
121057 2019-05-03 01:00:00  1.11730 -0.000179 -0.003217 -0.000537  0.004395   
121058 2019-05-03 02:00:00  1.11780  0.000447 -0.002145 -0.000447  0.004214   
121059 2019-05-03 03:00:00  1.11750 -0.000268 -0.002770 -0.000984  0.002957   
121060 2019-05-03 04:00:00  1.11750  0.000000 -0.002949 -0.002770  0.002778   
121061 2019-05-03 05:00:00  1.11740 -0.000089 -0.002770 -0.003306  0.003047   
121062 2019-05-03 06:00:00  1.11690 -0.000448 -0.002236 -0.003843  0.002869   
121063 2019-05-03 07:00:00  1.11720  0.000269 -0.003752 -0.004287  0.003228   
121064 2019-05-03 08:00:00  1.11620 -0.000895 -0.004202 -0.005183  0.003051   
121065 2019-05-03 09:00:00  1.11680  0.000537 -0.003040 -0.005180  0.003049   
121066 2019-05-03 10:00:00  1.11655 -0.000224 -0.003